In [3]:
import os,datetime,shutil,csv,lyricsgenius
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
client_access_token = 'IIQIH6uR1-KQ-fH3F1oPkdIVkRSLrbvVZ8H-yi6muIsxDUSWf22jjSSIe-Adq4UL'
genius = lyricsgenius.Genius(client_access_token)
genius

In [5]:
genius.remove_section_headers = True
genius.excluded_terms = ["(Remix)", "(Live)", "(Radio Edit)"]

In [6]:
POP_ARTISTS = {
    "genre": "Pop",
    "artists": [
        "Justin Bieber", "Adele", "Drake", "The Weeknd", "Taylor Swift", "Selena Gomez",
        "One Direction", "Shawn Mendes", "Meghan Trainor", "Ellie Goulding", "Rachel Platten",
        "Charlie Puth", "Demi Lovato", "Ariana Grande", "Ed Sheeran", "Alessia Cara",
        "Sam Smith", "Coldplay", "Maroon 5", "Beyoncé", "Nicki Minaj", "Justin Timberlake", "John Legend",
        "Lana Del Rey", "Rihanna", "Lady Gaga", "Katy Perry", "Miley Cyrus", "Sia", "Calvin Harris",
        "Robin Thicke", "Camila Cabello", "Michael Jackson", "Elton John", "Whitney Houston",
        "Madonna", "Bruno Mars", "Mariah Carey", "Harry Styles", "Marshmello",
        "P!nk", "Imagine Dragons", "Christina Aguilera", "Prince", "Stevie Wonder", "Normani", "Britney Spears",
        "Nickelback", "Rascal Flatts", "Ne-Yo", "Sean Paul", "The Pussycat Dolls", "James Blunt", "The Fray",
        "Kelly Clarkson", "The Black Eyed Peas", "Natasha Bedingfield", "Fergie",
        "Jamie Foxx", "Shakira", "Nelly", "T-Pain", "Kesha", "Snow Patrol", "Daniel Powter",
        "Janet Jackson", "Celine Dion", "R. Kelly", "Monica", "Toni Braxton", "Brandy", "Rod Stewart",
        "Jennifer Lopez", "Spice Girls", "Seal", "Backstreet Boys", "George Michael", "Amy Grant",
        "Ace of Base", "Soul Decision", "5 Seconds of Summer", "M2M", "Ricky Martin", "Steps",
        "Sugar Ray", "Natalia Oreiro"
    ]}
RAP_ARTISTS = {
    "genre": "Rap",
    "artists": [
        "Eminem", "Kendrick Lamar", "Big Shaq", "Cardi B", "Travis Scott", "Logic",
        "Kanye West", "Lil Uzi Vert", "Fetty Wap", "Post Malone", "JAY-Z", "Lil Wayne", "Snoop Dogg",
        "2Pac", "J. Cole", "50 Cent", "T.I.", "Dr. Dre", "Migos", "Future", "OutKast", "Busta Rhymes",
        "A$AP Rocky", "Ice Cube", "Rick Ross", "2 Chainz", "Wu-Tang Clan", "Juice Wrld", "Young Thug",
        "Big Sean", "Ghostface Killah", "Donald Glover & Danny Pudi", "Jeezy", "DMX", "Chance the Rapper", "Rakim",
        "LL Cool J", "Wiz Khalifa", "Chris Brown", "N.W.A", "Scarface", "XXXTENTACION", "The Notorious B.I.G.", "Nas",
        "André 3000", "Childish Gambino", "Nate Dogg", "Joyner Lucas", "The Game",
        "Ludacris", "Kid Cudi", "Redman", "Lauryn Hill", "KRS-One", "Big Pun", "Pusha T", "Hopsin", "ScHoolboy Q",
        "Meek Mill", "Raekwon", "Xzibit", "Slick Rick", "Denzel Curry", "MF DOOM", "Missy Elliott",
        "Big Daddy Kane", "Lupe Fiasco", "Gucci Mane", "Warren G", "RZA", "Q-Tip", "GZA", "B.o.B",
        "Twista", "21 Savage", "Big Boi", "Lil Dicky", "Prodigy of Mobb Deep", "NF", "Proof", "Talib Kweli",
        "Ski Mask the Slump God", "Obie Trice", "G-Eazy", "Yelawolf", "Earl Sweatshirt", "Chuck D", "MC Ren", "YG",
        "DaBaby"
    ]}
COUNTRY_ARTISTS = {
    "genre": "Country",
    "artists": [
        "Johnny Cash", "George Strait", "Dolly Parton", "Willie Nelson", "Garth Brooks", "Waylon Jennings",
        "Alan Jackson", "Merle Haggard", "George Jones", "Loretta Lynn", "Reba McEntire", "Carrie Underwood",
        "Shania Twain", "Patsy Cline", "Hank Williams", "Kenny Rogers", "Miranda Lambert",
        "Brad Paisley", "Vince Gill", "Dixie Chicks", "Buck Owens", "Tim McGraw", "Randy Travis", "Charley Pride",
        "Faith Hill", "Conway Twitty", "Tammy Wynette", "Emmylou Harris", "Kris Kristofferson", "Ray Price",
        "Keith Urban", "Glen Campbell", "Alabama", "Toby Keith", "Blake Shelton", "Linda Ronstadt",
        "Brooks & Dunn", "Dwight Yoakam", "John Denver", "Alison Krauss", "Jerry Jeff Walker", "Chris LeDoux",
        "Lucinda Williams", "Crystal Gayle", "The Judds", "Jamey Johnson", "Bill Anderson", "Eric Church",
        "Steve Earle", "Townes Van Zandt", "Lynn Anderson", "The Statler Brothers", "Rosanne Cash", "Patty Loveless",
        "Marty Stuart", "Asleep at the Wheel", "Lee Ann Womack", "Merle Travis", "Jessi Colter",
        "Vern Gosdin", "Connie Smith", "Guy Clark", "Tanya Tucker", "Roy Acuff", "Billy Joe Shaver",
        "The Carter Family", "Jimmie Rodgers", "Eddy Arnold", "Roger Miller",
        "Lefty Frizzell", "Ernest Tubb", "The Louvin Brothers", "Tom T. Hall", "Bill Monroe",
        "Flatt & Scruggs", "John Prine", "Cody Johnson", "Chris Stapleton", "Luke Combs", "Midland",
        "Cole Swindell", "Brantley Gilbert", "Chris Janson", "LANco", "Jon Pardi", "Dylan Scott", "Brett Young",
        "Brothers Osborne", "Old Dominion", "Dustin Lynch", "Michael Ray", "Aaron Watson", "Chris Lane", "Kane Brown"
    ]}
ROCK_ARTISTS = {
    "genre": "Rock",
    "artists": [
        "The Beatles", "The Rolling Stones", "Led Zeppelin", "Pink Floyd", "Queen", "David Bowie", "Elvis Presley",
        "AC/DC", "Guns N Roses", "Fleetwood Mac", "Metallica", "Bob Dylan", "Aerosmith", "Black Sabbath", "U2",
        "Bruce Springsteen", "The Who", "Paul McCartney", "The Doors", "Chuck Berry", "Ozzy Osbourne",
        "The Beach Boys", "Nirvana", "Eagles", "Van Halen", "Eric Clapton",
        "The Police", "Lynyrd Skynyrd", "Rush", "Deep Purple", "Red Hot Chili Peppers", "Pearl Jam",
        "Dire Straits", "Journey", "The Clash", "Radiohead", "The Kinks", "ZZ Top", "Foo Fighters", "Green Day",
        "Iron Maiden", "Ramones", "Genesis", "R.E.M.", "John Lennon", "Bon Jovi", "Neil Young", "Boston",
        "Def Leppard", "Scorpions", "Heart", "Billy Joel", "Kiss", "Jethro Tull", "Santana", "Steve Miller Band", "Yes",
        "Janis Joplin", "Talking Heads", "Foreigner", "Buddy Holly", "Chicago", "The Velvet Underground",
        "Alice Cooper", "Soundgarden", "Grateful Dead", "Steely Dan", "The Smashing Pumpkins", "Styx",
        "The Cure", "Blondie", "Alice in Chains", "Bad Company", "The Band", "The Yardbirds", "Jefferson Airplane",
        "Oasis", "The Cars", "George Harrison", "Sex Pistols", "Supertramp", "The Moody Blues", "Cheap Trick",
        "Linkin Park", "Rage Against the Machine", "The Jimi Hendrix Experience", "Greta Van Fleet",
        "Panic! At The Disco", "PVRIS", "​twenty one pilots", "Mumford & Sons", "Fall Out Boy",

    ]}
REGGAE_ARTISTS = {
    "genre": "Reggae",
    "artists": [
        "Bob Marley & The Wailers", "Jimmy Cliff", "Peter Tosh", "Burning Spear", "Ruth Brown",
        "Bunny Wailer", "Gregory Isaacs", "Ziggy Marley", "UB40", "Buju Banton", "Black Uhuru", "Desmond Dekker",
        "Beres Hammond", "Chronixx", "Shaggy", "Capleton", "Max Romeo", "Rita Marley",
        "Freddie McGregor", "Damian Marley", "Israel Vibration", "Steel Pulse", "Beenie Man", "Jah Cure", "Gentleman",
        "Stephen Marley", "Maxi Priest", "Luciano", "The Abyssinians", "Sizzla", "Bounty Killer",
        "Lucky Dube", "Alton Ellis", "John Holt", "Protoje", "The Upsetters", "Tarrus Riley"
    ]}

In [12]:
ARTISTS_BY_GENRE = {
    "Pop": POP_ARTISTS,
    "Rap": RAP_ARTISTS,
    "Country": COUNTRY_ARTISTS,
    "Rock": ROCK_ARTISTS,
    "Reggae": REGGAE_ARTISTS}

In [14]:
def make_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path

def save_dataset(path, input, append=True):
    if append:
        with open(path, 'a+', encoding='utf_8_sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(input)
    else:
        with open(path, 'w+', encoding='utf_8_sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(input)
    f.close()

def load_dataset(dataset_path):
    with open(dataset_path, encoding='utf_8_sig') as f:
        f = csv.reader(f)
        output = []
        for line in tqdm(f):
            # Output      (genre,   artist,   year,   album,  song_name, lyrics)
            output.append((line[0], line[1], line[2], line[3], line[4], line[5]))
    return output

In [10]:
save_dir = make_dir("datasets/")
save_path = os.path.join(save_dir, "genius_lyrics_v2.csv")

In [56]:
# 抓取Pop類型，並儲存到genius_lyrics_v2-Pop.csv
key = 'Pop'
lst = ARTISTS_BY_GENRE[key]["artists"][66:] # [66:]上一階段跑的結果
for artist in lst:
    data = []
    artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")
    if artist_obj is not None:
        for i in range(artist_obj.num_songs):
            song = artist_obj.songs[i]
            if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
                data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
        save_dataset(os.path.join(save_dir, "genius_lyrics_v2-Pop.csv"), data, append=True)

Searching for songs by Celine Dion...

Changing artist name to 'Céline Dion'
Song 1: "My Heart Will Go On"
Song 2: "How Does a Moment Last Forever"
Song 3: "Ashes"
Song 4: "Because You Loved Me"
Song 5: "The Power of Love"
Song 6: "It’s All Coming Back to Me Now"
Song 7: "Lying Down"
Song 8: "All By Myself"
Song 9: "Beauty and the Beast (Duet)"
Song 10: "Imperfections"
Song 11: "Pour que tu m’aimes encore"
Song 12: "Recovering"
Song 13: "Courage"
Song 14: "That’s the Way It Is"
Song 15: "If That’s What It Takes"
Song 16: "Parler à mon père"
Song 17: "I Love You"
"I’m Alive" is not valid. Skipping.
Song 18: "A New Day Has Come"
Song 19: "I Surrender"
Song 20: "Lovers Never Die"
Song 21: "The Prayer (duet With Andrea Bocelli) edit"
Song 22: "Encore un soir"
Song 23: "Flying on My Own"
Song 24: "Perfect Goodbye"
Song 25: "To Love You More"
Song 26: "Think Twice"
Song 27: "Falling In Love Again"
Song 28: "The Show Must Go On"
Song 29: "Goodbye’s (The Saddest Word)"
Song 30: "I’m Your Angel

Song 1: "I Wanna Be Down"
"I Wanna Be Down (Remix)" is not valid. Skipping.
Song 2: "Missing You"
Song 3: "Have You Ever?"
Song 4: "Put It Down"
Song 5: "Brokenhearted"
Song 6: "Best Friend"
Song 7: "Sittin’ Up In My Room"
Song 8: "Moesha Theme Song"
Song 9: "Almost Doesn’t Count"
Song 10: "Angel in Disguise"
Song 11: "I Don’t Care"
Song 12: "Top of the World"
Song 13: "Surprise Ending"
Song 14: "Full Moon"
Song 15: "Right Here (Departed)"
Song 16: "Long Distance"
Song 17: "Baby Mama"
Song 18: "Who Is She 2 U"
Song 19: "Talk About Our Love"
Song 20: "When You Touch Me"
Song 21: "Afrodisiac"
Song 22: "Baby"
Song 23: "True"
Song 24: "Freedom Rings"
Song 25: "What About Us?"
Song 26: "Everything I Do (I Do It For You)"
Song 27: "Nothing"
Song 28: "Piano Man"
Song 29: "Beggin & Pleadin"
"Deliver Me" is not valid. Skipping.
Song 30: "Tomorrow"
Song 31: "Never Say Never"
Song 32: "I Dedicate (Part I)"
Song 33: "Put That on Everything"
Song 34: "Come a Little Closer"
Song 35: "Camouflage"
Son

Song 1: "Kiss From a Rose"
Song 2: "Crazy"
Song 3: "Every Time I’m with You"
Song 4: "Fly Like an Eagle"
Song 5: "Stand By Me"
Song 6: "Love’s Divine"
Song 7: "Violet"
Song 8: "Killer"
Song 9: "A Change Is Gonna Come"
Song 10: "Love"
Song 11: "Prayer for the Dying"
Song 12: "Secret"
Song 13: "Amazing"
Song 14: "Daylight Savings"
Song 15: "It’s a Man’s Man’s Man’s World"
Song 16: "Future Love Paradise"
Song 17: "Don’t Cry"
Song 18: "Do You Ever"
Song 19: "Touch"
Song 20: "Padded Cell"
Song 21: "Human Beings"
Song 22: "I’ve Been Loving You Too Long"
Song 23: "Half A Heart"
Song 24: "The Big Love Has Died"
Song 25: "Life On The Dance Floor"
Song 26: "Bring it On"
Song 27: "Waiting for You"
Song 28: "Monascow"
Song 29: "We Don’t Need Another Hero"
Song 30: "Anyone Who Knows What Love Is"
Song 31: "Newborn Friend"
Song 32: "Deep Water"
Song 33: "Crazy (Acoustic Version)"
"Weight In Gold (Live with Seal)" is not valid. Skipping.
Song 34: "Dreaming in Metaphors"
Song 35: "Smile"
Song 36: "I C

Song 15: "Always Have, Always Will"
Song 16: "Dancer in a Daydream"
Song 17: "Love in December"
Song 18: "All for You"
Song 19: "Unspeakable"
Song 20: "Waiting for Magic"
Song 21: "Ravine"
Song 22: "Beautiful Morning"
Song 23: "My Deja Vu"
Song 24: "Angel Eyes"
Song 25: "Hallo Hallo"
Song 26: "Strange Ways"
Song 27: "Blooming 18"
Song 28: "All That She Wants (Banghra Version)"
"Happy Nation (Remix)" is not valid. Skipping.
Song 29: "Münchhausen (Just Chaos!)"
Song 30: "Tokyo Girl"
Song 31: "Da Capo"
Song 32: "Break My Stride"
Song 33: "All That She Wants (12 Inch Version)"
Song 34: "Hey Darling"
Song 35: "C’est la Vie (Always 21)"
Song 36: "Whispers in Blindness"
Song 37: "Love For Sale"
Song 38: "Fashion Party"
Song 39: "Wonderful Life"
Song 40: "Show Me Love"
Song 41: "Don’t Stop"
Song 42: "My Mind (Mindless Mix)"
Song 43: "Megamix - long version"
Song 44: "Wave Wet Sand"
Song 45: "Experience Pearls"
Song 46: "Perfect World"
Song 47: "Don’t Go Away"
Song 48: "Whenever You’re Near Me"

Song 28: "Dear Santa"
Song 29: "After The Love Has Gone"
Song 30: "You’ll Be Sorry"
Song 31: "Love’s Got A Hold On My Heart"
Song 32: "Dancing With A Broken Heart"
Song 33: "Merry Xmas Everybody"
Song 34: "When I Said Goodbye"
Song 35: "Stomp"
Song 36: "Thank Abba For The Music"
Song 37: "Better The Devil You Know"
Song 38: "Since You Took Your Love Away"
Song 39: "I Know Him So Well"
Song 40: "A Love To Last"
Song 41: "It’s The Way U Make Me Feel"
Song 42: "Stay With Me"
Song 43: "Baby Don’t Dance"
Song 44: "Love U More"
Song 45: "You’re Everything That Matters To Me"
Song 46: "This Heart Will Love Again"
Song 47: "Happy Go Lucky"
Song 48: "I Surrender"
Song 49: "Learn To Love Again"
Song 50: "Loves Got A Hold Of My Heart"
Song 51: "Make It Easy On Me"
Song 52: "It May Be Winter Outside"
Song 53: "Wouldn’t Hurt So Bad"
Song 54: "My Best Friends Girl"
Song 55: "I Think It’s Love"
Song 56: "Paradise Lost"
Song 57: "Dancing Queen"
Song 58: "Buzzz"
Song 59: "Light Up the World"
Song 60: "

In [62]:
# 抓取Rock類型，並儲存到genius_lyrics_v2-Rock.csv
key = 'Rock'
lst = ARTISTS_BY_GENRE[key]["artists"][18:] # [18:]上一階段跑的結果
for artist in lst:
    data = []
    artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")
    if artist_obj is not None:
        for i in range(artist_obj.num_songs):
            song = artist_obj.songs[i]
            if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
                data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
        save_dataset(os.path.join(save_dir, "genius_lyrics_v2-Rock.csv"), data, append=True)

Searching for songs by The Doors...

Song 1: "Riders on the Storm"
Song 2: "The End"
Song 3: "Light My Fire"
Song 4: "People Are Strange"
Song 5: "L.A. Woman"
Song 6: "Break On Through (To the Other Side)"
Song 7: "The Crystal Ship"
Song 8: "Touch Me"
Song 9: "Roadhouse Blues"
Song 10: "When the Music’s Over"
Song 11: "The Soft Parade"
Song 12: "Love Her Madly"
Song 13: "Hello, I Love You"
Song 14: "Soul Kitchen"
"Riders on the Storm (Fredwreck Remix)" is not valid. Skipping.
Song 15: "Five to One"
Song 16: "Peace Frog"
Song 17: "Alabama Song (Whisky Bar)"
Song 18: "Love Me Two Times"
Song 19: "Moonlight Drive"
Song 20: "Love Street"
Song 21: "Strange Days"
Song 22: "Waiting for the Sun"
Song 23: "Celebration of the Lizard King"
Song 24: "Back Door Man"
Song 25: "The Unknown Soldier"
Song 26: "The WASP (Texas Radio and the Big Beat)"
Song 27: "End of the Night"
Song 28: "Been Down So Long"
Song 29: "Twentieth Century Fox"
Song 30: "Spanish Caravan"
Song 31: "Blue Sunday"
Song 32: "Not 

Song 35: "Jesus Doesn’t Want Me for a Sunbeam"
Song 36: "Very Ape"
Song 37: "Radio Friendly Unit Shifter"
Song 38: "Love Buzz"
Song 39: "Negative Creep"
Song 40: "Moist Vagina"
Song 41: "Oh, Me"
Song 42: "Paper Cuts"
"Sappy (Live At Pine Street Theatre February 9, 1990)" is not valid. Skipping.
Song 43: "Old Age"
Song 44: "Sappy"
Song 45: "Downer"
Song 46: "Been a Son"
Song 47: "Moist Vagina (Rio de Janeiro Demo, January 1993)"
Song 48: "Dive"
Song 49: "Scoff"
Song 50: "Mr. Moustache"
Song 51: "Gallons of Rubbing Alcohol Flow Through the Strip"
Song 52: "Verse Chorus Verse"
Song 53: "Marigold"
Song 54: "Do Re Mi"
Song 55: "Aero Zeppelin"
Song 56: "Big Cheese"
Song 57: "Swap Meet"
Song 58: "Even In His Youth"
Song 59: "Sifting"
Song 60: "D-7"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by Eagles...

Song 1: "Hotel California"
Song 2: "Take It Easy"
Song 3: "Desperado"
Song 4: "One of These Nights"
Song 5: "Life in the Fast Lane"
Song 6: "New Kid in

Song 54: "Shadows in the Rain"
Song 55: "Behind My Camel"
Song 56: "The Other Way of Stopping"
Song 57: "De Do Do Do, De Da Da Da (Spanish Version)"
Song 58: "Fall Out"
Song 59: "Dead End Job"
Song 60: "Hungry For You"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by Lynyrd Skynyrd...

Song 1: "Sweet Home Alabama"
Song 2: "Free Bird"
Song 3: "Simple Man"
Song 4: "That Smell"
Song 5: "Tuesday’s Gone"
Song 6: "Gimme Three Steps"
Song 7: "The Ballad of Curtis Loew"
Song 8: "Workin’ for MCA"
Song 9: "I Ain’t the One"
Song 10: "Saturday Night Special"
Song 11: "The Needle and the Spoon"
Song 12: "Call Me the Breeze"
Song 13: "I Need You"
Song 14: "What’s Your Name"
Song 15: "Gimme Back My Bullets"
Song 16: "All I Can Do is Write About It"
Song 17: "God & Guns"
Song 18: "Things Goin’ On"
Song 19: "Don’t Ask Me No Questions"
Song 20: "Searching"
Song 21: "On the Hunt"
Song 22: "That’s How I Like It"
Song 23: "Red White and Blue"
Song 24: "Mississippi Kid"


Song 44: "Not For You"
Song 45: "Glorified G"
Song 46: "Lightning Bolt"
Song 47: "W.M.A."
Song 48: "Future Days"
Song 49: "Retrograde"
Song 50: "Hail Hail"
Song 51: "Smile"
Song 52: "Pendulum"
Song 53: "Leash"
Song 54: "Rats"
Song 55: "I Got Id"
Song 56: "Light Years"
Song 57: "Present Tense"
Song 58: "Buckle Up"
Song 59: "Getaway"
Song 60: "Never Destination"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by Dire Straits...

Song 1: "Money for Nothing"
Song 2: "Sultans of Swing"
Song 3: "Romeo and Juliet"
Song 4: "Brothers in Arms"
Song 5: "Walk of Life"
Song 6: "Tunnel Of Love"
Song 7: "Why Worry?"
Song 8: "Telegraph Road"
Song 9: "Your Latest Trick"
Song 10: "So Far Away"
Song 11: "Lady Writer"
Song 12: "Wild West End"
Song 13: "Down to the Waterline"
Song 14: "Six Blade Knife"
Song 15: "On Every Street"
Song 16: "Heavy Fuel"
Song 17: "Private Investigations"
Song 18: "Skateaway"
Song 19: "Ride Across the River"
Song 20: "The Man’s Too Strong"
Son

Song 16: "Better Things"
Song 17: "Destroyer"
Song 18: "Big Sky"
Song 19: "Father Christmas"
Song 20: "David Watts"
Song 21: "Come Dancing"
Song 22: "Dedicated Follower of Fashion"
Song 23: "Days"
Song 24: "Tired of Waiting for You"
Song 25: "Animal Farm"
Song 26: "Village Green"
Song 27: "Celluloid Heroes"
Song 28: "People Take Pictures of Each Other"
Song 29: "Death of a Clown"
Song 30: "Do You Remember Walter?"
Song 31: "Yes Sir, No Sir"
Song 32: "I’m Not Like Everybody Else"
Song 33: "Alcohol"
Song 34: "I’m A Lover Not A Fighter"
Song 35: "All of My Friends Were There"
Song 36: "Nothin’ in the World Can Stop Me Worryin’ ’Bout That Girl"
Song 37: "Arthur"
Song 38: "Powerman"
Song 39: "Last of the Steam-Powered Trains"
Song 40: "Sitting by the Riverside"
Song 41: "Australia"
Song 42: "Johnny Thunder"
Song 43: "Some Mother’s Son"
Song 44: "Get Back in Line"
Song 45: "Phenomenal Cat"
Song 46: "Mr. Churchill Says"
Song 47: "Black Messiah"
Song 48: "20th Century Man"
Song 49: "Harry Rag"

...

Song 45: "My Only One"
Song 46: "Easy On My Soul"
Song 47: "Early In The Morning"
Song 48: "Abandoned And Alone"
Song 49: "Call On Me"
Song 50: "Sweet Lil’ Sister"
"Live for the Music" is not valid. Skipping.
Song 51: "Dance With The Devil"
Song 52: "That Girl"
Song 53: "Walk Through Fire"
Song 54: "Here Comes Trouble"
Song 55: "Do Right By Your Woman"
Song 56: "Passing Time"
Song 57: "Painted Face"
Song 58: "Morning Sun"
Song 59: "Fame And Fortune"
Song 60: "I Don’t Care"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by The Band...

Song 1: "The Weight"
Song 2: "The Night They Drove Old Dixie Down"
Song 3: "Up on Cripple Creek"
Song 4: "Atlantic City"
Song 5: "I Shall Be Released"
Song 6: "Ophelia"
Song 7: "Acadian Driftwood"
Song 8: "It Makes No Difference"
Song 9: "The Shape I’m In"
Song 10: "Tears of Rage"
Song 11: "Chest Fever"
Song 12: "Across the Great Divide"
Song 13: "Whispering Pines"
Song 14: "To Kingdom Come"
Song 15: "King Harvest (Has 

Song 14: "Since You’re Gone"
Song 15: "Why Can’t I Have You"
Song 16: "Heartbeat City"
Song 17: "Hello Again"
Song 18: "It’s All I Can Do"
Song 19: "Dangerous Type"
Song 20: "Tonight She Comes"
Song 21: "Candy-O"
Song 22: "Double Life"
Song 23: "I’m Not The One"
Song 24: "I’m in Touch with Your World"
Song 25: "Don’t Cha Stop"
Song 26: "You Are the Girl"
Song 27: "Panorama"
Song 28: "Since I Held You"
Song 29: "Shoo Be Doo"
Song 30: "It’s Not The Night"
Song 31: "Gimme Some Slack"
Song 32: "Don’t Tell Me No"
Song 33: "Stranger Eyes"
Song 34: "Everything You Say"
Song 35: "Lust for Kicks"
Song 36: "Looking for Love"
Song 37: "Night Spots"
Song 38: "Got a Lot on My Head"
Song 39: "Cruiser"
Song 40: "Misfit Kid"
Song 41: "A Dream Away"
Song 42: "You Can’t Hold On Too Long"
Song 43: "I Refuse"
Song 44: "I’m in Touch With Your World (demo)"
Song 45: "You Wear Those Eyes"
Song 46: "Take Me Now"
Song 47: "Sad Song"
Song 48: "Soon"
Song 49: "Victim of Love"
Song 50: "Double Trouble"
Song 51: "

Song 32: "The Dream"
Song 33: "For My Lady"
Song 34: "Lovely to See You"
Song 35: "Dr. Livingstone, I Presume"
Song 36: "The Afternoon"
Song 37: "22,000 Days"
Song 38: "Go Now"
Song 39: "Watching and Waiting"
Song 40: "Emily’s Song"
Song 41: "Never Comes the Day"
Song 42: "Om"
Song 43: "Departure"
Song 44: "A Simple Game"
Song 45: "Have You Heard, Part 2"
Song 46: "I Know Your Out There Somewhere"
Song 47: "Long Summer Days"
Song 48: "Visions of Paradise"
Song 49: "Have You Heard? (Part 1)"
Song 50: "New Horizons"
Song 51: "You Can Never Go Home"
Song 52: "Nights In White Satin - Single"
Song 53: "December Snow"
Song 54: "The Word"
Song 55: "Candle of Life"
Song 56: "Had to Fall in Love"
Song 57: "Lost in a Lost World"
"One More Time to Live" is not valid. Skipping.
Song 58: "Steppin’ in a Slide Zone"
Song 59: "Are You Sitting Comfortably"
Song 60: "Nights In White Satin (The Night) - Album Version / Incl. “Late Lament”"

Reached user-specified song limit (60).
Done. Found 60 songs.
Se

Song 1: "Black Smoke Rising"
Song 2: "Highway Tune"
Song 3: "Safari Song"
Song 4: "Flower Power"
Song 5: "When the Curtain Falls"
Song 6: "Edge of Darkness"
Song 7: "Talk On the Street"
Song 8: "Age of Man"
Song 9: "Anthem"
Song 10: "Lover, Leaver (Taker, Believer)"
Song 11: "Watching Over"
Song 12: "Always There"
Song 13: "A Change Is Gonna Come"
Song 14: "Meet On The Ledge"
Song 15: "Mountain of the Sun"
Song 16: "Brave New World"
Song 17: "The New Day"
Song 18: "Lover, Leaver"
Song 19: "The Cold Wind"
Song 20: "You’re the One"
Song 21: "Rolling In The Deep (Recorded At Spotify Studios NYC)"
Song 22: "Black Flag Exposition"
Song 23: "Thunderstomp"
Song 24: "Lover Leaver Taker Believer Jam"
Song 25: "The Jean Genie"
Song 26: "That’s All Right Mama"
Song 27: "Down to the River"
"Lover, Leaver (Taker, Believer) (Live)" is not valid. Skipping.
Song 28: "Standing On"
Done. Found 28 songs.
Searching for songs by Panic! At The Disco...

Changing artist name to 'Panic! at the Disco'
Song 1: 

Song 23: "This Ain’t a Scene, It’s An Arms Race"
Song 24: "Fourth of July"
Song 25: "Disloyal Order of Water Buffaloes"
Song 26: "Sunshine Riptide"
Song 27: "Save Rock and Roll"
Song 28: "Jet Pack Blues"
Song 29: "Miss Missing You"
Song 30: "Young Volcanoes"
Song 31: "Bishops Knife Trick"
Song 32: "I Don’t Care"
Song 33: "The Mighty Fall"
Song 34: "American Beauty/American Psycho"
Song 35: "The Drunk History of Fall Out Boy"
Song 36: "Our Lawyer Made Us Change the Name of This Song So We Wouldn’t Get Sued"
Song 37: "I’m Like a Lawyer with the Way I’m Always Trying to Get You Off (Me & You)"
Song 38: "“The Take Over, the Breaks Over”"
Song 39: "Novocaine"
Song 40: "Where Did the Party Go"
Song 41: "Headfirst Slide into Cooperstown on a Bad Bet"
Song 42: "A Little Less Sixteen Candles, a Little More “Touch Me”"
Song 43: "Rat a Tat"
Song 44: "Hum Hallelujah"
Song 45: "G.I.N.A.S.F.S."
Song 46: "She’s My Winona"
Song 47: "27"
Song 48: "Thriller"
Song 49: "​w.a.m.s."
Song 50: "Twin Skeleton’

In [96]:
# 抓取Rap類型，並儲存到genius_lyrics_v2-Rap.csv
key = 'Rap'
lst = ARTISTS_BY_GENRE[key]["artists"][87:] #[87:]上一階段跑的結果
for artist in lst:
    data = []
    artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")
    if artist_obj is not None:
        for i in range(artist_obj.num_songs):
            song = artist_obj.songs[i]
            if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
                data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
        save_dataset(os.path.join(save_dir, "genius_lyrics_v2-Rap.csv"), data, append=True)

Searching for songs by MC Ren...

"Compton’s N the House (Remix)" is not valid. Skipping.
"Ice Cube & The Notorious B.I.G. - Hello Vs. Party & Bullshit (Matoma Remix)" is not valid. Skipping.
Song 1: "Ruthless For Life"
Song 2: "Final Frontier"
"Fuck Tha Police (Live)" is not valid. Skipping.
Song 3: "Same Old Shit"
Song 4: "Who In The Fuck"
Song 5: "Fuck What Ya Heard"
Song 6: "Mad Scientist"
Song 7: "Rebel Music"
Song 8: "Behind the Scenes"
"N.Y. State Of Mind (Live)" is not valid. Skipping.
Song 9: "Comin’ After You"
Song 10: "Keep It Real"
"Rebel Music (Remix)" is not valid. Skipping.
Song 11: "Bitch Made Nigga Killa"
Song 12: "You Wanna Fuck Her"
Song 13: "Mayday on the Frontline"
Song 14: "Right Up My Alley"
Song 15: "Shock of the Hour"
Song 16: "Mr. Fuck Up"
Song 17: "Kizz My Black Azz"
Song 18: "One False Move"
Song 19: "Hounddogz"
Song 20: "Do You Believe"
Song 21: "Attack on Babylon"
"Fuck Tha Police Remix" is not valid. Skipping.
Song 22: "All Bullshit Aside"
"We Want Eazy (

In [107]:
# 抓取Country類型，並儲存到genius_lyrics_v2-Country.csv
key = 'Country'
lst = ARTISTS_BY_GENRE[key]["artists"][78:] #[78:]上一階段跑的結果
for artist in lst:
    data = []
    artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")
    if artist_obj is not None:
        for i in range(artist_obj.num_songs):
            song = artist_obj.songs[i]
            if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
                data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
        save_dataset(os.path.join(save_dir, "genius_lyrics_v2-Country.csv"), data, append=True)

Searching for songs by Luke Combs...

Song 1: "Beautiful Crazy"
Song 2: "When It Rains It Pours"
Song 3: "Dive"
Song 4: "She Got the Best of Me"
Song 5: "One Number Away"
Song 6: "I Know She Ain’t Ready"
Song 7: "Hurricane"
Song 8: "Better Together"
Song 9: "Even Though I’m Leaving"
Song 10: "Must’ve Never Met You"
Song 11: "Can I Get an Outlaw?"
Song 12: "Used to You"
Song 13: "What You See Is What You Get"
Song 14: "Does To Me"
Song 15: "Houston, We Got a Problem"
Song 16: "This One’s For You"
Song 17: "Lonely One"
Song 18: "Dear Today"
Song 19: "Nothing Like You"
Song 20: "I Got Away with You"
Song 21: "A Long Way"
Song 22: "All Over Again"
Song 23: "Let the Moonshine"
Song 24: "Sheriff You Want To"
Song 25: "Reasons"
Song 26: "Be Careful What You Wish For"
Song 27: "Lovin’ on You"
Song 28: "The Way She Rides"
Song 29: "Beer Never Broke My Heart"
Song 30: "Blue Collar Boys"
Song 31: "New Every Day"
Song 32: "Moon Over Mexico"
Song 33: "Six Feet Apart"
Song 34: "1, 2 Many"
Song 35: "

Song 2: "Born to Love You"
Song 3: "Trouble Maker"
"Long Live Tonight" is not valid. Skipping.
Song 4: "We Do"
Song 5: "Pick You Up"
Song 6: "So Long (I Do)"
Song 7: "Hallelujah Nights"
Song 8: "Win You Over"
Song 9: "Singin’ at the Stars"
Song 10: "Rival"
Song 11: "Middle of the Night"
Song 12: "What I See"
Song 13: "Born To Love You (RISE Recording)"
Done. Found 13 songs.
Searching for songs by Jon Pardi...

Song 1: "Night Shift"
Song 2: "Dirt On My Boots"
Song 3: "Heartache Medication"
Song 4: "Head Over Boots"
Song 5: "Heartache On The Dance Floor"
Song 6: "She Ain’t In It"
Song 7: "Can’t Turn You Down"
Song 8: "Nobody Leaves a Girl Like That"
Song 9: "Over My Head"
Song 10: "Starlight"
Song 11: "When I’ve Been Drinkin’"
Song 12: "Old Hat"
Song 13: "Cowboy Hat"
Song 14: "Don’t Blame It On Whiskey"
Song 15: "Love Her Like She’s Leaving"
Song 16: "Lucky Tonight"
Song 17: "Ain’t Always the Cowboy"
Song 18: "Out of Style"
Song 19: "Call Me Country"
Song 20: "California Sunrise"
Song 21

Song 19: "Little Town Livin’"
Song 20: "Here We Come"
Song 21: "Your Plan"
Song 22: "Middle Of Nowhere"
Song 23: "Sun Don’t Go Down on That"
Song 24: "What You Wanna Hear"
Song 25: "Why Not Tonight"
Song 26: "Dirt Road"
Song 27: "Party Song"
Song 28: "Hurricane"
Song 29: "The World Ain’t Yours and Mine"
Song 30: "Workin’ On You"
Song 31: "She Cranks My Tractor"
Song 32: "Your Daddy’s Boots"
Song 33: "Sing It To Me"
Song 34: "Country Star"
Song 35: "Halo"
Song 36: "To The Sky"
Song 37: "Waiting"
Song 38: "Old Country Song"
Song 39: "All Night"
Song 40: "Sittin’ Pretty"
Song 41: "Wild In Your Smile"
Song 42: "She Wants A Cowboy"
Song 43: "After Party"
Song 44: "Dancing In The Headlights"
Song 45: "Right Where We Want It"
Song 46: "World To Me"
Song 47: "American Prayer"
Song 48: "Yeah Yeah Yeah"
Song 49: "Rock You Sweet"
Song 50: "Last Lap"
Song 51: "Name On It"
Song 52: "Unwind It"
"She Cranks My Tractor (Club Remix)" is not valid. Skipping.
Song 53: "Cowboys and Angels (PO clean edit)"

In [ ]:
# 抓取Reggae類型，並儲存到genius_lyrics_v2-Reggae.csv
key = 'Reggae'
lst = ARTISTS_BY_GENRE[key]["artists"] #[78:]上一階段跑的結果
for artist in lst:
    data = []
    artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")
    if artist_obj is not None:
        for i in range(artist_obj.num_songs):
            song = artist_obj.songs[i]
            if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
                data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
        save_dataset(os.path.join(save_dir, "genius_lyrics_v2-Reggae.csv"), data, append=True)

Searching for songs by Bob Marley & The Wailers...

Song 1: "No Woman, No Cry"
Song 2: "Redemption Song"
Song 3: "Three Little Birds"
Song 4: "One Love / People Get Ready (Medley)"
Song 5: "Could You Be Loved"
Song 6: "Buffalo Soldier"
Song 7: "Waiting in Vain"
Song 8: "Is This Love"
Song 9: "Jamming"
Song 10: "War"
Song 11: "Natural Mystic"
Song 12: "Exodus"
Song 13: "Rat Race"
Song 14: "Easy Skanking"
Song 15: "Satisfy My Soul"
Song 16: "Time Will Tell"
Song 17: "Them Belly Full (But We Hungry)"
Song 18: "Who the Cap Fit"
Song 19: "Turn Your Lights Down Low"
Song 20: "Positive Vibration"
Song 21: "Crazy Baldheads"
Song 22: "Zimbabwe"
Song 23: "Iron Lion Zion"
Song 24: "Coming in from the Cold"
Song 25: "Pimper’s Paradise"
Song 26: "Bad Card"
Song 27: "Running Away"
Song 28: "Forever Loving Jah"
Song 29: "One Drop"
Song 30: "The Heathen"
Song 31: "Africa Unite"
Song 32: "So Much Things To Say"
Song 33: "Talkin’ Blues"
Song 34: "Punky Reggae Party"
Song 35: "Guiltiness"
Song 36: "Babyl

Song 39: "Jack O’ Diamonds"
Done. Found 39 songs.
Searching for songs by Bunny Wailer...

Song 1: "Cool Runnings"
Song 2: "Ballroom Floor"
Song 3: "Dreamland"
Song 4: "Blackheart Man"
Song 5: "Bald Head Jesus"
Song 6: "Fighting Against Conviction"
Song 7: "Mellow mood"
Song 8: "Ready When You Ready"
Song 9: "Rise and Shine"
Song 10: "Here In Jamaica"
Song 11: "Love Fire"
Song 12: "Liberation"
Song 13: "Rule This Land"
Song 14: "Botha the Mosquito"
Song 15: "Rasta Man"
Song 16: "Hypocrite"
Song 17: "The Oppressed Song"
Song 18: "Armagideon (Armagedon)"
Song 19: "Roots Radics Rockers Reggae"
"Kinky Reggae (Live)" is not valid. Skipping.
Song 20: "Rockers"
Song 21: "Fig Tree"
Song 22: "This Train"
Song 23: "I’m The Toughest"
Song 24: "Reincarnated Souls"
Song 25: "Want to Come Home"
Song 26: "Walk The Proud Land"
Song 27: "Dash Wey the Vial"
Song 28: "Bide Up"
Song 29: "Serious Thing"
Song 30: "Follow Fashion Monkey"
Song 31: "Three Little Birds"
Song 32: "I gotta keep on moving"
Song 33:

Song 52: "Paid Not Played"
Song 53: "Friends for Life"
Song 54: "Optimistic Soul"
Song 55: "No More Misty Days"
Song 56: "Love Dem Bad (feat. Red Rat)"
Song 57: "Dickie"
Song 58: "It’s All Over"
Song 59: "Can You Play Some More (feat. Beres Hammond)"
Song 60: "All Will Be Fine (feat. Sons and Daughters Choir)"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by Black Uhuru...

Song 1: "Sinsemilla"
Song 2: "Guess Who’s Coming to Dinner"
Song 3: "Sponji Reggae"
Song 4: "Puff She Puff"
Song 5: "Abortion"
Song 6: "Darkness"
Song 7: "Black Uhuru Anthem"
Song 8: "Plastic Smile"
Song 9: "Youth of Eglington"
Song 10: "Shine Eye Gal"
Song 11: "Great Train Robbery"
Song 12: "System"
Song 13: "Utterance"
Song 14: "Happiness"
Song 15: "Peace and Love"
Song 16: "Push Push"
Song 17: "Eye Market"
Song 18: "Fit You Haffe Fit"
Song 19: "Leaving to Zion"
Song 20: "General Penitentiary"
Song 21: "Chill Out"
Song 22: "What Is Life?"
Song 23: "Satan Army Band"
Song 24: "Ti

Song 29: "Keep’n It Real"
Song 30: "Hot Shot"
Song 31: "Chica Bonita"
Song 32: "Fight This Feeling"
Song 33: "When She Loves Me"
"Come Around (B-Real Remix)" is not valid. Skipping.
Song 34: "Lonely Lover"
Song 35: "Would You Be"
Song 36: "Ultimatum"
Song 37: "Dance & Shout"
Song 38: "Use Me"
Song 39: "Not Fair"
Song 40: "Luv Me, Luv Me"
Song 41: "Feel the Rush"
Song 42: "Freaky Girl"
Song 43: "Lucky Day"
Song 44: "Why You Treat Me So Bad"
Song 45: "Hey Love"
Song 46: "Repent"
Song 47: "Shaggy, Where Are You?"
Song 48: "You"
Song 49: "Fly High"
"It Wasn’t Me - Ehrling Remix" is not valid. Skipping.
Song 50: "Leave Me Alone"
Song 51: "Hey Sexy Lady (Original Sting International Mix)"
Song 52: "Sexy Body Girls"
Song 53: "I need your love (te quiero mas)"
Song 54: "Fired Up"
Song 55: "Body Good"
Song 56: "Day Oh"
Song 57: "Full Control"
Song 58: "Sugarcane"
Song 59: "What’s Love"
Song 60: "Get My Party On"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs 

Song 48: "We’re Gonna Make It"
Song 49: "Give Dem Some Way"
Song 50: "Born to Be Wild"
Song 51: "Searching (So Much Bubble)"
Song 52: "Carnal Mind"
Song 53: "Catch a Fire"
"Shoot Out (Remix)" is not valid. Skipping.
Song 54: "Love and Inity"
Song 55: "Wanted (Just Ain’t The Same)"
Song 56: "Stand Up Jamrock"
Song 57: "Holiday"
Song 58: "Love and Unity"
Song 59: "Trouble"
Song 60: "Halfway Tree"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by Israel Vibration...

Song 1: "My Master’s Will"
Song 2: "Back Staba"
Song 3: "Natty Dread"
Song 4: "Rudeboy Shufflin"
Song 5: "Vultures"
Song 6: "There Is No End"
Song 7: "Herb Is The Healing"
Song 8: "Falling Angels"
Song 9: "Cool And Calm"
Song 10: "Why Worry"
Song 11: "Middle East"
Song 12: "We a de rasta"
Song 13: "The Same Song"
Song 14: "Licks and Kicks"
Song 15: "Ball of Fire"
Song 16: "Ambush"
Song 17: "Jah Is The Way"
Song 18: "Lift Up Your Conscience"
Song 19: "Why you so craven"
Song 20: "Rude boy sh

Song 27: "Children of Tomorrow"
Song 28: "Good Old Days"
Song 29: "Tranquility"
Song 30: "Walk Away"
Song 31: "Jah Ina Your Life"
Song 32: "Journey To Jah"
Song 33: "No Solidarity - MTV Unplugged"
Song 34: "Another Melody"
Song 35: "Empress"
Song 36: "I Got To Go"
Song 37: "Different Places"
Song 38: "Caan Hold Us Down"
Song 39: "Fire Ago Bun Dem"
Song 40: "Fast Forward"
Song 41: "Ich trink auf dich (aus „Sing meinen Song, Vol. 4“)"
Song 42: "Road of Life"
Song 43: "Unconditional Love"
"Jah jah never fail - live" is not valid. Skipping.
Song 44: "Love Chant"
Song 45: "Beat to my Melody (aus „Sing meinen Song, Vol. 4“)"
Song 46: "What She Deserves"
Song 47: "Ina Time Like Now"
Song 48: "Shake Away (aus „Sing meinen Song, Vol. 4“)"
Song 49: "Soulfood"
Song 50: "All That You Had"
Song 51: "See Dem Coming"
Song 52: "The Journey"
Song 53: "Hold On Strong"
Song 54: "No Doubt About It"
Song 55: "Ina Different Time"
Song 56: "Sin City"
Song 57: "Moment Of Truth"
Song 58: "Nothin’a Change"
Song

Song 26: "Black Woman And Child"
"Give Me A Try (Remix)" is not valid. Skipping.
Song 27: "Holding Firm"
Song 28: "Perfect One"
Song 29: "Love You for Who You Are"
Song 30: "Marijuana"
Song 31: "Beautiful Day [The Overstanding]"
Song 32: "To the Point"
Song 33: "Give It To Dem"
Song 34: "Bless Up"
Song 35: "Haven’t I Told You"
Song 36: "Give Them the Ride"
Song 37: "Mockeries And Phrase"
Song 38: "Ganja"
Song 39: "Ain’t Gonna See Us Fall"
Song 40: "No Time to Gaze"
Song 41: "Where are you running to"
Song 42: "Taking Over"
Song 43: "Dem Ah Wonder"
Song 44: "Love Is Divine"
"Hostile Gospel Pt. 2 (Deliver Me)" is not valid. Skipping.
Song 45: "No Other Like Jah"
Song 46: "Like Mountain"
Song 47: "She’s Loving"
Song 48: "One Away"
Song 49: "Show Me"
Song 50: "The World"
Song 51: "Talk All They Want"
Song 52: "Blackness"
Song 53: "Give Thanks"
Song 54: "Good Morning"
"Solid as a Rock (Never Scared Remix)" is not valid. Skipping.
Song 55: "Can’t Cool Can’t Quench"
Song 56: "Really and truly

Song 2: "Rasta Love"
Song 3: "Kingston Be Wise"
Song 4: "Resist Not Evil"
Song 5: "No Lipstick"
Song 6: "Blood Money (BLXXD MONEY)"
Song 7: "Protection"
Song 8: "No Guarantee"
Song 9: "Bout Noon"
Song 10: "Sudden Flight"
Song 11: "Not Another Word"
Song 12: "This Is NOT A Marijuana Song"
Song 13: "Like This"
Song 14: "A Matter of Time"
Song 15: "Shot By Love"
Song 16: "Dread"
Song 17: "Mind of a King"
"Underdog (Remix)" is not valid. Skipping.
Song 18: "Flames"
Song 19: "Roll"
Song 20: "The Flame"
Song 21: "Who Dem a Program"
Song 22: "Criminal"
Song 23: "Camera Show"
Song 24: "Stylin’"
Song 25: "Hail Ras Tafari"
Song 26: "I&I"
Song 27: "Flight Plans"
Song 28: "Glad You’re Home"
Song 29: "Can’t Feel No Way"
Song 30: "Take Control"
Song 31: "Arguments"
Song 32: "Used To Be My Life"
Song 33: "Who Can You Call"
Song 34: "Lessons"
Song 35: "Hold You Now"
Song 36: "Love Gone Cold"
Song 37: "Sandra Foster"
Song 38: "Black Cinderella"
Song 39: "Wrong Side Of The Law"
Song 40: "Answer To Your 

In [108]:
key = 'Reggae'
lst = ARTISTS_BY_GENRE[key]["artists"]
print(lst)

['Bob Marley & The Wailers', 'Jimmy Cliff', 'Peter Tosh', 'Burning Spear', 'Ruth Brown', 'Bunny Wailer', 'Gregory Isaacs', 'Ziggy Marley', 'UB40', 'Buju Banton', 'Black Uhuru', 'Desmond Dekker', 'Beres Hammond', 'Chronixx', 'Shaggy', 'Capleton', 'Max Romeo', 'Rita Marley', 'Freddie McGregor', 'Damian Marley', 'Israel Vibration', 'Steel Pulse', 'Beenie Man', 'Jah Cure', 'Gentleman', 'Stephen Marley', 'Maxi Priest', 'Luciano', 'The Abyssinians', 'Sizzla', 'Bounty Killer', 'Lucky Dube', 'Alton Ellis', 'John Holt', 'Protoje', 'The Upsetters', 'Tarrus Riley']


In [31]:
all_songs = load_dataset(save_path)
print("\n\nThe total amount of scraped songs: {}".format(len(all_songs)))

1it [00:00, 1002.46it/s]


IndexError: list index out of range

In [32]:
all_artists = []
for key in ARTISTS_BY_GENRE.keys():
    all_artists += ARTISTS_BY_GENRE[key]["artists"]

In [33]:
specified_duplicate_artists = set([x for x in all_artists if all_artists.count(x) > 1])
print("The following {} artists were duplicate: {}".format(len(specified_duplicate_artists), specified_duplicate_artists))

unique_found_songs = set([lyrics[5] for lyrics in all_songs])
print("There are duplicate songs {} found.".format(len(all_songs)-len(unique_found_songs)))

all_artists = set(all_artists)
unique_found_artists = set([lis[1] for lis in all_songs])
not_found_artists = all_artists.difference(unique_found_artists)
print("The scraped data does not contain songs from the following {} artists:\n{}".format(len(not_found_artists), not_found_artists))

The following 0 artists were duplicate: set()


NameError: name 'all_songs' is not defined

# Extra

In [17]:
artist = 'Drake'
artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")

Searching for songs by Drake...

Song 1: "God’s Plan"
Song 2: "In My Feelings"
Song 3: "Hotline Bling"
Song 4: "One Dance"
Song 5: "Hold On, We’re Going Home"
Song 6: "Know Yourself"
Song 7: "Back to Back"
Song 8: "All Me"
Song 9: "Fake Love"
Song 10: "From Time"
Song 11: "0 to 100 / The Catch Up"
Song 12: "The Motto"
Song 13: "Started from the Bottom"
Song 14: "Pound Cake / Paris Morton Music 2"
Song 15: "HYFR"
Song 16: "Marvin’s Room"
"Versace (Remix)" is not valid. Skipping.
Song 17: "Nice For What"
Song 18: "Passionfruit"
Song 19: "Child’s Play"
Song 20: "Energy"
Song 21: "Too Good"
Song 22: "Furthest Thing"
Song 23: "Jungle"
"My Way (Remix)" is not valid. Skipping.
Song 24: "Over My Dead Body"
Song 25: "Worst Behavior"
Song 26: "Summer Sixteen"
Song 27: "The Language"
Song 28: "Girls Love Beyoncé"
Song 29: "How Bout Now"
Song 30: "Too Much"
Song 31: "Shot for Me"
Song 32: "Legend"
Song 33: "Portland"
Song 34: "Nonstop"
Song 35: "Controlla"
Song 36: "Gyalchester"
Song 37: "U With M

In [29]:
data = []

In [30]:
if artist_obj is not None:
    for i in range(29,31):
        song = artist_obj.songs[i]
        if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
            data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
    save_dataset(os.path.join(save_dir, "genius_lyrics_v2-1.csv"), data, append=True)  
print(data)

[('Pop', 'Drake', '2011', 'Take Care', 'Shot for Me', 'I can see it in your eyes, you\'re angry\nRegret got shit on what you\'re feelin\' now\nMad cause he ain\'t like me\nOh you mad cause nobody ever did it like me\nAll the care I would take\nAll the love that we made\nNow you\'re trying to find somebody to replace what I gave to you\nIt\'s a shame you didn\'t keep it\nAlisha, Catya, I know that you gon\' hear this\nI\'m the man, yeah I said it\nBitch, I\'m the man, don\'t you forget it\nThe way you walk, that\'s me\nThe way you talk, that\'s me\nThe way you\'ve got your hair up, did you forget that\'s me?\nAnd the voice in your speaker right now that\'s me\nThat\'s me, and the voice in your ear\nThat\'s me, can\'t you see\nThat I made it? Yeah, I made it\nFirst I made you who you are and then I made it\nAnd you\'re wasted with your latest\nYeah, I\'m the reason why you always getting faded\n\nTake a shot for me (Oh-oh-oh-oh, oh-oh-oh-oh)\nTake a shot for me (Oh-oh-oh-oh, oh-oh-oh-oh)

In [34]:
# 抓取metadata
for key in ARTISTS_BY_GENRE.keys():
    for artist in ARTISTS_BY_GENRE[key]["artists"]:
        data = []
        artist_obj = genius.search_artist(artist, max_songs=60, sort="popularity")
        if artist_obj is not None:
            for i in range(artist_obj.num_songs):
                song = artist_obj.songs[i]
                if None not in (song.artist, song.year, song.album, song.title, song.lyrics):
                    data.append((ARTISTS_BY_GENRE[key]["genre"], song.artist, song.year[:4], song.album, song.title, song.lyrics))
            #save_path = os.path.join(save_dir, "genius_lyrics_v2-Rap.csv")
            save_dataset(save_path, data, append=True)

Searching for songs by Justin Bieber...

"Despacito (Remix)" is not valid. Skipping.
Song 1: "Love Yourself"
Song 2: "Sorry"
Song 3: "Yummy"
Song 4: "As Long As You Love Me"
Song 5: "Baby"
Song 6: "What Do You Mean?"
Song 7: "Intentions"
Song 8: "Confident"
Song 9: "I’ll Show You"
Song 10: "Beauty and a Beat"
Song 11: "Mark My Words"
Song 12: "Purpose"
Song 13: "Boyfriend"
Song 14: "No Sense"
Song 15: "Company"
Song 16: "The Feeling"
Song 17: "No Pressure"
Song 18: "All That Matters"
Song 19: "Never Say Never"
Song 20: "Hold Tight"
Song 21: "Come Around Me"
Song 22: "Forever"
Song 23: "Heartbreaker"
Song 24: "Life Is Worth Living"
Song 25: "Right Here"
Song 26: "Get Me"
Song 27: "Habitual"
Song 28: "Mistletoe"
Song 29: "Changes"
"Boyfriend (Remix)" is not valid. Skipping.
"Foreign (Remix)" is not valid. Skipping.
Song 30: "We Are"
Song 31: "Been You"
Song 32: "Hard 2 Face Reality"
Song 33: "Nothing Like Us"
Song 34: "All Around Me"
Song 35: "E.T.A."
Song 36: "All Bad"
Song 37: "Get Use

Song 8: "Call It What You Want"
Song 9: "Delicate"
"I Don’t Wanna Live Forever" is not valid. Skipping.
Song 10: "Style"
Song 11: "Cruel Summer"
Song 12: "Gorgeous"
Song 13: "Don’t Blame Me"
Song 14: "The Man"
Song 15: "I Did Something Bad"
Song 16: "Dress"
Song 17: "The Archer"
Song 18: "Bad Blood"
Song 19: "Miss Americana & The Heartbreak Prince"
Song 20: "London Boy"
Song 21: "Getaway Car"
Song 22: "Cornelia Street"
Song 23: "King of My Heart"
Song 24: "I Forgot That You Existed"
Song 25: "Daylight"
Song 26: "Wildest Dreams"
Song 27: "New Year’s Day"
Song 28: "Death by a Thousand Cuts"
Song 29: "So It Goes..."
Song 30: "Paper Rings"
Song 31: "Love Story"
Song 32: "This Is Why We Can’t Have Nice Things"
Song 33: "False God"
Song 34: "I Think He Knows"
Song 35: "Soon You’ll Get Better"
Song 36: "Afterglow"
Song 37: "Dancing with Our Hands Tied"
Song 38: "All Too Well"
Song 39: "Shake It Off"
Song 40: "Clean"
Song 41: "It’s Nice to Have a Friend"
Song 42: "Out of the Woods"
Song 43: "W

Song 59: "Funk"
Song 60: "Throwback Love"

Reached user-specified song limit (60).
Done. Found 60 songs.
Searching for songs by Ellie Goulding...

Song 1: "Love Me Like You Do"
Song 2: "On My Mind"
Song 3: "Burn"
Song 4: "Still Falling for You"
Song 5: "Lights"
Song 6: "Army"
Song 7: "Anything Could Happen"
Song 8: "How Long Will I Love You"
Song 9: "Flux"
Song 10: "Sixteen"
Song 11: "Something in the Way You Move"
Song 12: "Hanging On"
Song 13: "Beating Heart"
Song 14: "Vincent"
Song 15: "My Blood"
Song 16: "Explosions"
Song 17: "Love Me Like You Do (Fifty Shades Freed Version)"
Song 18: "Figure 8"
Song 19: "Your Song"
Song 20: "Goodness Gracious"
Song 21: "Don’t Need Nobody"
Song 22: "Dead In the Water"
"Times Like These (BBC Radio 1 Stay Home Live Lounge)" is not valid. Skipping.
Song 23: "Aftertaste"
Song 24: "I Know You Care"
Song 25: "Tessellate"
Song 26: "Lost and Found"
Song 27: "Heal"
"Close to Me (Red Velvet Remix)" is not valid. Skipping.
Song 28: "Codes"
Song 29: "I Do What

TypeError: argument of type 'NoneType' is not iterable